# Point of interest

This notebook presumes that you have completed the previous jupyter notebook and have obtained a "business table" containing all of the businesses of a specific type/class(ie. gyms). This notebook creates a distance matrix(note that it takes some time, so the code version is recommended as it is faster and the current progress. Another limitation of this notebook is that it takes only a subset of the data needed to calculate the distance matrix. It is incomplete. To remove this limitation, skip the sampling step. There are two steps; creating the distance matrix using Euclidean distances, and creating variables(as well as joining them to the dataset). 

## Distance Matrix creation

This distance matrix features the id's of the various gyms in the region as the row id's. The id's of the real-estate sold are the names of the columns. The values are simply the Euclidean distances between each of those two points. To start we load the two datasets as well as the libraries used.

In [9]:
KingCounty <- read.csv("C://Users//silvi//THESIS//kc_house_data.csv")
BusinessData <- read.csv("C://Users//silvi//THESIS//active.csv")
getwd()
library(geosphere)
library(dplyr)
library(tidyr)


[1] "C:/Users/silvi/THESIS"

[1] 20

Do not run the part below should you wish to obtain the full dataset. 

In [ ]:
#Get the first 20 rows of each
KingCounty <-KingCounty[c(1:20),]
BusinessData <-BusinessData[c(1:20),]
nrow(KingCounty)

In [ ]:
#The distance between the first house and all of the restaurants

#Full value
#length()
dataframe <- data.frame(matrix(ncol=nrow(KingCounty),nrow=nrow(BusinessData)))

for(d in 1:nrow(KingCounty)){
      
      
      #The inner loop is done, this will fill out the distances between the first house and ALL of the business objects
      distances=vector()
      #Full length
      #
      for(i in 1:nrow(BusinessData)){
            DistFirstHouse <- distm(
                  y=c(KingCounty$long[d],
                      KingCounty$lat[d]),
                  x=c(BusinessData$longitude[i],                             BusinessData$latitude[i]),
                  fun=distGeo)
            distances[i] <- DistFirstHouse 
      }
      DistanceData[d] <- distances
      #print(paste0("Progress=",d," out of ",nrow(KingCounty)))
      ##This should show the progress bar better
}

names(dataframe) <-KingCounty$id 
DistanceData$BusinessID <- BusinessData$id
DistanceData$BusinessRating <- BusinessData$rating
DistanceData$BusinessReviewCount <- BusinessData$review_count

This completes the creation of the distance matrix. 

In [ ]:
head(DistanceData)

# #Feature Creation

The code below will create two new features/variables. One will be the count of objects within a 100m range while the other will be their average rating. 
Some of the real-estate sold is does not have any gyms within a 100m range. In the case of a simple count the result will be a 0. In the case of the average rating the result will not be a number due to the division by zero. Due to that, a value of  -1000 is imputed.  

In [11]:
#Primary key "BusinessID" in both but we will not connect with it but only get the data out
#1:1 relation
#Get the Business Data
BusinessData <- DistanceData[,c("BusinessID","BusinessRating","BusinessReviewCount")]
#Get the Distance Data cause we need a diff algorithm
#If you have 21617 variables here it probably has "X". remove it. You have to end up with 21613
DistanceData <- DistanceData[,-which(names(DistanceData) %in% c("BusinessRating","BusinessReviewCount","BusinessID","X"))]

####################
#KingCounty blw 100
###################
#First split, searching for real estate that has business distances within 100m and the id's of these businesses
blw100 <- data.frame(matrix(ncol=21613,nrow=1))
for (i in 1:length(names(DistanceData))){
   blw100[i] <- sum(DistanceData[,i]<100)
      print(i)
      
}
KingCounty$blw100 <- t(blw100)



#This is the average rating
blw100AvgRating <- data.frame(matrix(ncol=21613, nrow=1))
names(blw100AvgRating) <- names(DistanceData)
index <- logical()
for (i in 1:length(names(DistanceData))){
   index <-ifelse(DistanceData[,i]<100,TRUE,FALSE)
   blw100AvgRating[i] <- mean(BusinessData$BusinessRating[index])
   index <- logical()
   
}

#This transformation turns the NaN's into -1000
x <- as.data.frame(t(blw100AvgRating))
#Turning the NA's to zeroes
a <- matrix(t(x))
a[is.nan(a)] <- -1000

KingCounty$blw100AvgRating <-a


Warning message in file(file, "rt"):
"cannot open file 'D://THESIS data//Distances//active_distances.csv': No such file or directory"

ERROR: Error in file(file, "rt"): cannot open the connection
